# Medallion Architecture

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Delta Lake & Lakehouse
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 16.4 LTS lub nowszy (zalecane: 17.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard lub **Serverless Compute** (zalecane)
- **Zależności**: Wykonany notebook `00_setup.ipynb`
- **Czas realizacji**: ~60 minut

> **Uwaga (2025):** Serverless Compute jest teraz domyślnym trybem dla nowych workloadów.

## Wstęp teoretyczny

### Lakeflow - Platforma Data Ingestion

**Lakeflow** to zintegrowana platforma Databricks do zarządzania przepływem danych:

| Komponent | Opis | Use Case |
|-----------|------|----------|
| **COPY INTO** | Batch loading z cloud storage | Scheduled ETL, duże pliki |
| **Auto Loader** | Streaming z cloud storage | Continuous ingestion, małe pliki |
| **Lakeflow Connect** | Managed connectors do SaaS | Salesforce, SAP, Workday |
| **Lakeflow Pipelines** | Declarative ETL (DLT) | Bronze → Silver → Gold |

---

### Batch vs Streaming - Decision Matrix

| Cecha | Batch (COPY INTO) | Streaming (Auto Loader) |
|-------|-------------------|-------------------------|
| **Latency** | Minuty-Godziny | Sekundy-Minuty |
| **File size** | Duże (>1GB) | Małe (<100MB) |
| **Frequency** | Scheduled (hourly/daily) | Continuous |
| **Cost** | Niższy (on-demand) | Wyższy (always-on) |
| **Complexity** | Niski | Średni |
| **Idempotency** | ✅ Built-in | ✅ Checkpoint-based |
| **Schema evolution** | Manual | Automatic |

**Kiedy Batch (COPY INTO):**
- ✅ Dane przychodzą w scheduled intervals
- ✅ Duże pliki (> 1GB)
- ✅ Latency nie jest krytyczna
- ✅ Lower cost requirement

**Kiedy Streaming (Auto Loader):**
- ✅ Dane przychodzą ciągle (< 1h intervals)
- ✅ Potrzebujesz low latency (< 5 min)
- ✅ Małe pliki (< 100MB each)
- ✅ Real-time dashboards/analytics

## Izolacja per użytkownik

In [0]:
%run ../00_setup

## Konfiguracja

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime
import time

**Konfiguracja ścieżek i zmiennych:**

In [0]:
# Ustawienie domyślnego katalogu i schematu
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# === ŚCIEŻKI DO DANYCH BATCH ===
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
PRODUCTS_PARQUET = f"{DATASET_BASE_PATH}/products/products.parquet"

# === ŚCIEŻKI DO DANYCH STREAMING ===
STREAMING_SOURCE_PATH = f"{DATASET_BASE_PATH}/orders/stream"

# === ŚCIEŻKI TECHNICZNE ===
CHECKPOINT_BASE_PATH = f"{DATASET_BASE_PATH}/{raw_user}/lakeflow_checkpoints"
SCHEMA_BASE_PATH = f"{DATASET_BASE_PATH}/{raw_user}/lakeflow_schemas"
BAD_RECORDS_PATH = f"{DATASET_BASE_PATH}/{raw_user}/bad_records"

**Czyszczenie poprzednich uruchomień (dla demo):**

In [0]:
try:
    dbutils.fs.rm(CHECKPOINT_BASE_PATH, True)
    dbutils.fs.rm(SCHEMA_BASE_PATH, True)
    dbutils.fs.rm(BAD_RECORDS_PATH, True)
except:
    pass

**Weryfikacja konfiguracji:**

In [0]:
display(
    spark.createDataFrame([
        ("CATALOG", CATALOG),
        ("BRONZE_SCHEMA", BRONZE_SCHEMA),
        ("SILVER_SCHEMA", SILVER_SCHEMA),
        ("USER", raw_user),
        ("CUSTOMERS_CSV", CUSTOMERS_CSV),
        ("STREAMING_SOURCE_PATH", STREAMING_SOURCE_PATH)
    ], ["Variable", "Value"])
)

---

## Sekcja 1: COPY INTO - Batch Loading

**COPY INTO** to rekomendowana metoda batch ingestion:
- **Idempotency**: Automatyczne śledzenie przetworzonych plików
- **File tracking**: Tylko nowe pliki są ładowane przy ponownym uruchomieniu
- **Format support**: CSV, JSON, Parquet, Avro, ORC

### Przykład 1.1: COPY INTO z CSV

In [0]:
TABLE_CUSTOMERS = f"{BRONZE_SCHEMA}.customers_batch"

**Tworzenie target table:**

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {TABLE_CUSTOMERS} (
  customer_id STRING,
  first_name STRING,
  last_name STRING,
  email STRING,
  phone STRING,
  city STRING,
  state STRING,
  country STRING,
  registration_date DATE,
  customer_segment STRING,
  _ingestion_timestamp TIMESTAMP
) USING DELTA
COMMENT 'Customers data - Bronze layer'
""")

**Wykonanie COPY INTO:**

In [0]:
result = spark.sql(f"""
COPY INTO {TABLE_CUSTOMERS}
FROM (
  SELECT 
    customer_id,
    first_name,
    last_name,
    email,
    phone,
    city,
    state,
    country,
    TO_DATE(registration_date, 'yyyy-MM-dd') as registration_date,
    customer_segment,
    current_timestamp() as _ingestion_timestamp
  FROM '{CUSTOMERS_CSV}'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false')
COPY_OPTIONS ('mergeSchema' = 'true')
""")

display(result)

**Weryfikacja wyników:**

In [0]:
display(
    spark.createDataFrame([
        ("Table", TABLE_CUSTOMERS),
        ("Records", str(spark.table(TABLE_CUSTOMERS).count()))
    ], ["Metric", "Value"])
)

**Przykładowe dane:**

In [0]:
display(spark.table(TABLE_CUSTOMERS).limit(5))

### Przykład 1.2: Test idempotency

Uruchomienie COPY INTO ponownie **nie** doda duplikatów:

In [0]:
count_before = spark.table(TABLE_CUSTOMERS).count()

# Ponowne uruchomienie COPY INTO
spark.sql(f"""
COPY INTO {TABLE_CUSTOMERS}
FROM (
  SELECT 
    customer_id, first_name, last_name, email, phone,
    city, state, country,
    TO_DATE(registration_date, 'yyyy-MM-dd') as registration_date,
    customer_segment,
    current_timestamp() as _ingestion_timestamp
  FROM '{CUSTOMERS_CSV}'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false')
""")

count_after = spark.table(TABLE_CUSTOMERS).count()

**Porównanie:**

In [0]:
display(
    spark.createDataFrame([
        ("Before", count_before),
        ("After", count_after),
        ("Difference", count_after - count_before)
    ], ["State", "Count"])
)

---

## Sekcja 2: Auto Loader - Streaming Ingestion

**Auto Loader (cloudFiles)** to Databricks-managed streaming source:
- Automatyczne file discovery (file notifications)
- Incremental processing (tylko nowe pliki)
- Schema inference & evolution
- Checkpoint management
- Skaluje do milionów plików

### Przykład 2.1: Auto Loader z trigger `availableNow`

---

## Trigger Modes

**Trigger** określa jak często streaming query wykonuje micro-batches:

| Mode | Zachowanie | Use Case |
|------|------------|----------|
| `availableNow=True` | Przetworz wszystko → stop | Scheduled jobs ⭐ |
| `once=True` | Legacy (deprecated) | - |
| `processingTime="10s"` | Co 10 sekund | Real-time |
| `continuous="1s"` | Ultra-low latency | Experimental |

In [0]:
TARGET_TABLE_AL = f"{BRONZE_SCHEMA}.orders_autoloader"
CHECKPOINT_AL = f"{CHECKPOINT_BASE_PATH}/autoloader"
SCHEMA_AL = f"{SCHEMA_BASE_PATH}/autoloader"

spark.sql(f"DROP TABLE IF EXISTS {TARGET_TABLE_AL}")

**Konfiguracja Auto Loader readStream:**

In [0]:
df_autoloader = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", SCHEMA_AL)
    .option("cloudFiles.inferColumnTypes", "true")
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
    .load(STREAMING_SOURCE_PATH)
)

**Dodanie metadata columns:**

In [0]:
df_enriched = (df_autoloader
    .withColumn("_processing_time", F.current_timestamp())
    .withColumn("_source_file", F.input_file_name())
)

**Uruchomienie streaming z trigger `availableNow`:**

> `availableNow` - przetwarza wszystkie dostępne dane i zatrzymuje się (batch-like streaming)

In [0]:
query = (df_enriched.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", CHECKPOINT_AL)
    .trigger(availableNow=True)
    .toTable(TARGET_TABLE_AL)
)

query.awaitTermination()

**Wyniki Auto Loader:**

In [0]:
display(
    spark.createDataFrame([
        ("Records Loaded", str(spark.table(TARGET_TABLE_AL).count())),
        ("Source Files", str(spark.table(TARGET_TABLE_AL).select("_source_file").distinct().count()))
    ], ["Metric", "Value"])
)

**Inferred schema:**

In [0]:
spark.table(TARGET_TABLE_AL).printSchema()

**Przykładowe dane:**

In [0]:
display(spark.table(TARGET_TABLE_AL).limit(10))

---

## Sekcja 3: Rescued Data Column

**Rescued Data Column** to mechanizm Auto Loader do obsługi nieoczekiwanych danych:

| Scenariusz | Behavior |
|------------|----------|
| Nowe kolumny | Zapisane w `_rescued_data` |
| Niepasujące typy | Zapisane w `_rescued_data` |
| Malformed records | Zapisane w `_rescued_data` |

**Konfiguracja:**
```python
.option("cloudFiles.schemaEvolutionMode", "rescue")
```

**Tryby schemaEvolutionMode:**
- `addNewColumns`: Automatycznie dodaje nowe kolumny
- `rescue`: Nowe kolumny → `_rescued_data` JSON
- `failOnNewColumns`: Fail jeśli schema się zmieni
- `none`: Ignoruje nowe kolumny (ryzykowne!)

### Przykład 3.1: Auto Loader z rescue mode

In [0]:
TARGET_TABLE_RESCUE = f"{BRONZE_SCHEMA}.orders_rescued"
CHECKPOINT_RESCUE = f"{CHECKPOINT_BASE_PATH}/rescue"
SCHEMA_RESCUE = f"{SCHEMA_BASE_PATH}/rescue"

spark.sql(f"DROP TABLE IF EXISTS {TARGET_TABLE_RESCUE}")

**Definiujemy explicit schema (częściowy):**

In [0]:
# Celowo definiujemy tylko część kolumn - reszta trafi do _rescued_data
partial_schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("total_amount", DoubleType(), True)
])

**Auto Loader z rescue mode:**

In [0]:
df_rescue = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", SCHEMA_RESCUE)
    .option("cloudFiles.schemaEvolutionMode", "rescue")  # Rescue mode!
    .schema(partial_schema)  # Partial schema
    .load(STREAMING_SOURCE_PATH)
)

**Uruchomienie stream:**

In [0]:
query_rescue = (df_rescue.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", CHECKPOINT_RESCUE)
    .trigger(availableNow=True)
    .toTable(TARGET_TABLE_RESCUE)
)

query_rescue.awaitTermination()

**Schema z kolumną `_rescued_data`:**

In [0]:
spark.table(TARGET_TABLE_RESCUE).printSchema()

**Dane z rescued columns:**

In [0]:
display(
    spark.table(TARGET_TABLE_RESCUE)
    .limit(5)
)

---

## Sekcja 4: Error Handling

**Strategie obsługi błędów w COPY INTO:**

| Mode | Zachowanie |
|------|------------|
| `PERMISSIVE` | Parsuje co się da, błędy → `_corrupt_record` |
| `DROPMALFORMED` | Usuwa błędne rekordy |
| `FAILFAST` | Zatrzymuje na pierwszym błędzie |

**`badRecordsPath`** - zapisuje błędne rekordy do folderu dla późniejszej analizy.

### Przykład 4.1: Error handling z badRecordsPath

In [0]:
TABLE_ERRORS = f"{BRONZE_SCHEMA}.customers_with_validation"

**Tworzenie tabeli z `_corrupt_record`:**

In [0]:
spark.sql(f"DROP TABLE IF EXISTS {TABLE_ERRORS}")

spark.sql(f"""
CREATE TABLE {TABLE_ERRORS} (
  customer_id STRING,
  first_name STRING,
  last_name STRING,
  email STRING,
  phone STRING,
  city STRING,
  state STRING,
  country STRING,
  registration_date STRING,
  customer_segment STRING,
  _corrupt_record STRING,
  _ingestion_timestamp TIMESTAMP
) USING DELTA
""")

**Wczytywanie z error handling:**

In [0]:
df_with_errors = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("columnNameOfCorruptRecord", "_corrupt_record")
    .option("badRecordsPath", BAD_RECORDS_PATH)
    .schema("""
        customer_id STRING,
        first_name STRING,
        last_name STRING,
        email STRING,
        phone STRING,
        city STRING,
        state STRING,
        country STRING,
        registration_date DATE,
        customer_segment STRING,
        _corrupt_record STRING
    """)
    .load(CUSTOMERS_CSV)
    .withColumn("_ingestion_timestamp", F.current_timestamp())
)

#df_with_errors.write.mode("overwrite").saveAsTable(TABLE_ERRORS)

In [0]:
display(df_with_errors)

**Statystyki błędnych rekordów:**

In [0]:
%skip
dbutils.fs.ls(BAD_RECORDS_PATH)

In [0]:
%skip
df_bad_records = spark.read.format("json").load("/Volumes/ecommerce_platform_trainer/default/kion_datasets/trainer/bad_records/20251125T115221/bad_records/")

display(df_bad_records)

---

## Sekcja 5: Lakeflow Connect (Informacyjna)

**Lakeflow Connect** to managed SaaS integration bez pisania kodu:

### Wspierane źródła:
- Salesforce
- Workday
- Google Analytics
- HubSpot
- Stripe
- SAP
- Netsuite
- ServiceNow

### Kluczowe cechy:

| Cecha | Opis |
|-------|------|
| **Zero-code** | Konfiguracja przez UI |
| **Managed** | Automatyczne skalowanie |
| **CDC Support** | Change Data Capture |
| **Schema Evolution** | Automatyczne aktualizacje |
| **Unity Catalog** | Pełna integracja |

### Jak uruchomić:
1. Workspace → **Data** → **Lakeflow** → **Connect**
2. Wybierz connector (np. Salesforce)
3. Podaj credentials
4. Wybierz obiekty do synchronizacji
5. Ustaw schedule

### Porównanie metod ingestion:

| Metoda | Use Case |
|--------|----------|
| **COPY INTO** | Pliki w cloud storage (batch) |
| **Auto Loader** | Pliki w cloud storage (streaming) |
| **Lakeflow Connect** | Dane z systemów SaaS |
| **Lakeflow Pipelines** | Transformacje Bronze → Silver → Gold |

---

## Podsumowanie

### Co osiągnęliśmy:

✅ **COPY INTO** - Idempotent batch loading z CSV/JSON/Parquet

✅ **Auto Loader** - Streaming ingestion z file notifications

✅ **Rescued Data Column** - Obsługa nieoczekiwanych danych

✅ **Error Handling** - badRecordsPath i corrupt record handling

✅ **Lakeflow Connect** - Integracja z systemami SaaS

✅ **Trigger Modes** - availableNow vs processingTime

### Kluczowe wnioski:

| # | Zasada |
|---|--------|
| 1 | **COPY INTO** dla batch, **Auto Loader** dla streaming |
| 2 | Zawsze używaj **checkpointLocation** dla streaming |
| 3 | **Rescue mode** dla bezpiecznej obsługi zmian schematu |
| 4 | **availableNow** dla scheduled jobs (cost-effective) |
| 5 | **Lakeflow Connect** dla integracji SaaS (zero-code) |

### Następne kroki:

📚 **Kolejny Notebook:** `03_medallion_architecture.ipynb`
- Bronze → Silver → Gold pattern
- Data quality layers
- Transformacje i agregacje

---

## Cleanup (Opcjonalnie)

In [0]:
# Lista utworzonych tabel
created_tables = [
    "customers_batch",
    "orders_autoloader",
    "orders_rescued",
    "customers_with_validation",
    "orders_trigger_test"
]

**Weryfikacja utworzonych zasobów:**

In [0]:
results = []
for table in created_tables:
    full_table = f"{CATALOG}.{BRONZE_SCHEMA}.{table}"
    try:
        if spark.catalog.tableExists(full_table):
            count = spark.table(full_table).count()
            results.append((table, "EXISTS", str(count)))
        else:
            results.append((table, "NOT FOUND", "-"))
    except Exception as e:
        results.append((table, "ERROR", str(e)[:30]))

display(spark.createDataFrame(results, ["Table", "Status", "Records"]))

In [0]:
# Flaga cleanup
CLEANUP_ENABLED = False

**Wykonanie cleanup (jeśli włączone):**
> 💡 *Zalecane: Zostaw dane dla kolejnych notebooków!*

In [0]:
if CLEANUP_ENABLED:
    results = []
    for table in created_tables:
        full_table = f"{CATALOG}.{BRONZE_SCHEMA}.{table}"
        try:
            spark.sql(f"DROP TABLE IF EXISTS {full_table}")
            results.append((table, "DROPPED"))
        except Exception as e:
            results.append((table, f"ERROR: {str(e)[:30]}"))
    
    # Cleanup checkpoints
    try:
        dbutils.fs.rm(CHECKPOINT_BASE_PATH, True)
        results.append(("checkpoints", "REMOVED"))
    except:
        results.append(("checkpoints", "NOT FOUND"))
    
    display(spark.createDataFrame(results, ["Resource", "Status"]))
else:
    display(spark.createDataFrame([
        ("CLEANUP_ENABLED", "False"),
        ("Akcja", "Zmień na True aby usunąć zasoby")
    ], ["Setting", "Value"]))